In [11]:
from ase.io import read, write
from mace.calculators import mace_mp
from ase.build import surface
from ase.visualize import view
from ase.constraints import FixAtoms, UnitCellFilter
from ase.build import bulk
import numpy as np
from ase.optimize import LBFGS

x = read('/home/lana/Downloads/9007497.cif')
x.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(x)
qn = LBFGS(ucf, trajectory='bulk_rlx.traj')
qn.run(fmax=0.01)
E_x = x.get_potential_energy()
t = read('bulk_rlx.traj@:')
view(t)

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 15:07:04      -28.749845*       0.0015


<Popen: returncode: None args: ['/home/lana/ve/bin/python3', '-m', 'ase', 'g...>

In [38]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np


def cu2o_bulk():
  return read('bulk_rlx.traj')

b = cu2o_bulk()
b.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
E_bulk = b.get_potential_energy()


def cu2o111(n_slab, vacuum):
  x = cu2o_bulk()
  x.translate([0.1, 0.1, 0])
  slab = surface(x, (1,1,1), n_slab, vacuum=vacuum, periodic=True)  
  return slab 


n_layers = 3
slab = cu2o111(n_layers, 10)

#view(slab)
write('slab.xyz', slab)


slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')

E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')

#def STO_FCC111():
bottom_O_z = np.min(slab[slab.symbols=='O'].positions[:,2])
maskO=slab.positions[:, 2] <= bottom_O_z 
bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
STO_z = np.max(slab[slab.symbols=='O'].positions[:,2])
del slab[maskO]
mask2 = (slab.positions[:, 2] >= STO_z) & (slab.symbols=='O')
del slab[mask2]

#STO_FCC111()

mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))
qn = LBFGS(slab, trajectory='111slab.traj')
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
t = read('111slab.traj@:')
atoms = t[-1]
#view(t)

view(atoms)



   
E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}')


Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
n_layers=3 E_cleav=0.09781531101608756
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 16:06:15      -67.526475*       1.4236
LBFGS:    1 16:06:16      -67.597947*       1.1723
LBFGS:    2 16:06:16      -67.819203*       0.6779
LBFGS:    3 16:06:16      -67.962566*       0.6284
LBFGS:    4 16:06:17      -68.051696*       0.8730
LBFGS:    5 16:06:17      -68.092568*       0.8230
LBFGS:    6 16:06:17      -68.253886*       0.9669
LBFGS:    7 16:06:17      -68.282901*       0.5702
LBFGS:    8 16:06:18      -68.299035*       0.5603
LBFGS:    9 16:06